# Debugging traceroute failures caused by ACLs with Batfish

Network engineers are responsible for verifying that the ACLs in their networks are permitting and denying traffic as intended. This generally requires loading each ACL onto a lab device in order to test its behavior on a wide range of packets. Batfish makes it easy to see what each ACL will do with any packet, right down to the line of the ACL that matches it. We also provide a sanity check to ensure that every line in every ACL can match some packet that won't be matched by a previous line.

In this notebook, we will start by showing an example network where a traceroute is failing with because the flow is denied by an ACL
To debug this problem, we find which ACLs cause the deny action with the help of `testfilters` question, and later explore the full scope of the flows the problematic ACL affects with the `reachfilter` question.

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [1]:
# Import packages and load questions
%run startup.py

# Initialize a network and snapshot
NETWORK_NAME = "acl_debugging_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

/Users/victor/pybatfish/pybatfish/client/commands.py:47: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


In [2]:
bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

## Performing a traceroute

Let us test that host2 can be reached from `AS3`. The Batfish virtual traceroute question mimics the behavior of real traceroute, with additional capability of specifying ports/protocols and the start location of the trace.

#### Tip: Pull up a table of IP addresses to help fill in `src` and `dst` ip address
The IP Owners question achieves this nicely (we'll use [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/) here and throughout the notebook to filter results):

In [3]:
# Show IP Owners for host1 and devices in AS1
ip_owners = bfq.ipOwners().answer().frame()
ip_owners[(ip_owners.Hostname.isin(['as3core1', 'host2']))]

,Hostname,VRF,Interface,IP,Mask,Active
5,as3core1,default,GigabitEthernet2/0,90.90.90.1,24,True
20,as3core1,default,Loopback0,3.10.1.1,32,True
21,as3core1,default,GigabitEthernet1/0,3.0.1.2,24,True
22,host2,default,eth0,2.128.1.101,24,True
29,as3core1,default,GigabitEthernet0/0,3.0.2.2,24,True
40,as3core1,default,GigabitEthernet3/0,90.90.90.2,24,True


We ask the traceroute question using the start location of `as3core1`, on the Loopback0 interface, going to the `host2` (IP 2.128.1.101)

What we are interested in is the `results` column, indicating the fate of the flow (e.g., `ACCEPT` or `DENIED`)

In [4]:
display(bfq.traceroute(srcIpSpace='3.10.1.1', dst='2.128.1.101', traceStart='enter(as3core1[Loopback0])', dstPort=80).answer().frame().drop('paths', axis='columns'))

,node,dstIp,flow,numPaths,results
0,as3core1,2.128.1.101,as3core1ingressInterface: Loopback0->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,4,[DENIED_IN]


We see that the result is `DENIED_IN` leading us to believe that an input ACL on some interface is responsible for this flow being dropped.

Let us examine which ACLs in the network can affect the flow we just constructed using the `testfilters` question.

### Test Filters

The Test Filters question takes in the properties of a flow and a set of ACLs to test. The answer shows what each ACL will do to that flow and why.

To specify the flow, you must provide the source IP address `srcIp` and the destination `dst` (hostname or IP address). Optionally, you can also specify IP protocols, ports, TCP flags, ICMP codes, and other properties ([see documentation for details](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.testfilters)). The question will fill in any unspecified flow properties with default values.

The set of ACLs to examine can be narrowed down using these optional parameters:
- `nodeRegex` specifies the ACLs present on the set of nodes matching the given regex
- `filterRegex` specifies ACLs with names matching the given regex

If `nodeRegex` and `filterRegex` are left blank, Test Filters will give results for every ACL in the network.

In this case we ask the question for *all* ACLs, and filter the ouput to focus on lines that match and result in the `DENY` action.


In [5]:
fr = bfq.testfilters(dst='2.128.1.101', srcIp='3.10.1.1', dstPort=80).answer().frame()
# Filter the output
fr[(fr.action == 'DENY') & (fr.lineContent != 'no-match')]

,node,filterName,flow,action,lineNumber,lineContent,trace
0,host2,filter::INPUT,host2->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,1.0,default,"Flow denied by ACL named 'filter::INPUT', index 1: default"
10,host1,filter::INPUT,host1->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,2.0,default,"Flow denied by ACL named 'filter::INPUT', index 2: default"
13,as2border1,INSIDE_TO_AS1,as2border1->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,2.0,deny ip any any,"Flow denied by 'extended ipv4 access-list' named 'INSIDE_TO_AS1', index 2: deny ip any any"
18,as2border1,OUTSIDE_TO_INSIDE,as2border1->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,1.0,deny ip any host 2.128.1.101,"Flow denied by 'extended ipv4 access-list' named 'OUTSIDE_TO_INSIDE', index 1: deny ip any host 2.128.1.101"
21,as2dept1,RESTRICT_HOST_TRAFFIC_IN,as2dept1->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,1.0,deny ip any any,"Flow denied by 'extended ipv4 access-list' named 'RESTRICT_HOST_TRAFFIC_IN', index 1: deny ip any any"
23,as2border2,INSIDE_TO_AS3,as2border2->[3.10.1.1:0->2.128.1.101:80 proto: IP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,2.0,deny ip any any,"Flow denied by 'extended ipv4 access-list' named 'INSIDE_TO_AS3', index 2: deny ip any any"


As we can see, there are multiple ACLs that can drop our flow. Lets work from the destination outward and focus on `filter::INPUT` on `host2` for our initial analysis.
We know it matches our conditions because the flow was `DENIED_IN` and the filter is applied on `INPUT`.

## Exploration with reachfilter
Let's see if there is **anything** that filter permits, with the help of `reachfilter` question.
`reachfilter` allows you to explore full space of flows affected by an ACL and return a representative flow that matches some action/query (e.g., `permit`, `deny` or `matchLine X`). [See full documentation for details](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.reachfilter)

We constrain the set of nodes and ACL to analyze using `nodeSpecifierInput='host2'` and `filterRegex='filter::INPUT'`, thus scoping the query to only one filter.
Similarly to other questions, we specify source/destination IPs and the type of query we care about (in this case, `'permit'`).

In [6]:
bfq.reachfilter(src='3.10.1.1', dst='2.128.0.101', query='permit', nodeSpecifierInput='host2', filterRegex='filter::INPUT').answer().frame()

,node,filterName,flow,action,lineNumber,lineContent,trace
0,host2,filter::INPUT,host2->[3.10.1.1:0->2.128.0.101:22 proto: TCP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,PERMIT,0,-p tcp --dport 22 -j ACCEPT,"Flow permitted by ACL named 'filter::INPUT', index 0: -p tcp --dport 22 -j ACCEPT"


Here we see that the filter allows `SSH` connections to `host2` (port 22), and based on the syntax of the `lineContent` column, we can infer that it is an *iptables* rule. 

### Futher scoping
As a curiosity exercise, let us see if `filter::INPUT` permits anything else besides SSH. For this we can can ask "Show me what's permitted, that is **not** port 22?" with the help of `complementHeaderSpace` parameter.


In [8]:
bfq.reachfilter(dstPorts=[22], complementHeaderSpace=True, query='permit', nodeSpecifierInput='host2', filterRegex='filter::INPUT').answer().frame()

,node,filterName,flow,action,lineNumber,lineContent,trace


Great, we get back an empty result, telling us that only SSH is permitted to `host2`. That means we will need to punch a hole in the iptables rules on for HTTP traffic.

To sum up:
- We performed a virtual traceroute for an HTTP flow, finding it to be blocked
- We investigated which ACL/filters can drop that flow
- We concluded that a necessary change is to change iptables rules on host2 to allow HTTP traffic

## Get involved with the Batfish community

Thanks for checking out our ACL debugging examples! To get involved and learn more, check out the community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). We would love to talk with you about Batfish or your network!